In [14]:
import SimFunctions
import SimClasses
import SimRNG
import simpy
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy.stats import probplot, kstest
import pickle
from scipy.stats.distributions import norm, expon, erlang, lognorm, weibull_min, gamma
import statsmodels.api as sm
warnings.filterwarnings("ignore")
# fix random number seed
np.random.seed(1)


In [31]:
class BikeShareSystem:
    def __init__(self, env, Full_Error, Empty_Error, num_stations, num_docks_per_station, destination_id, num_steps):
        self.env = env
        self.Full_Error = Full_Error
        self.Empty_Error = Empty_Error
        self.stations = [simpy.Container(
            env, capacity=num_docks_per_station) for i in range(num_stations)]
        self.destination_id = destination_id
        self.current_step = 8
        self.num_steps = num_steps

    def rent_bike(self, station_id):
        # Request a bike from the station
        yield self.stations[station_id].get(1)

    def return_bike(self, station_id):
        # Return a bike to the station
        yield self.stations[station_id].put(1)

    # Define a customer that uses the bike share system
    def Destination(self, station_id):
        print("Destination")
        U = np.random.uniform(0, 1)
        self.destination_id = 1 if U > 1/2 else 0

    def Departure(self, station_id):
        print("Departure")
        yield self.env.timeout(5*np.random.lognormal(0.1, np.sqrt(0.066)))

    def Customer_Arrival(self, station_id):
        print("Customer_Arrival")
        # STATION EMPTY
        if (self.stations[station_id].level) <= 0:
            self.Empty_Error += 1
        else:
            yield self.rent_bike(station_id)
            yield self.Destination(station_id)
            yield self.Departure(station_id)
            yield self.Bike_Arrival(self.destination_id)

    def Bike_Arrival(self, destination_id):
        print("Bike Arrival")

        # yield self.env.timeout(np.random.exponential(5))
        # STATION FULL
        if self.stations[destination_id].capacity <= (self.stations[destination_id].level):
            self.Full_Error += 1

        if (self.stations[destination_id].level) < self.stations[destination_id].capacity:
            yield self.return_bike(destination_id)

    def Retrial(self, station_id):  # retrial function
        print("Retrial")
        # yield self.env.timeout(np.random.exponential(5))

        # STATION FULL
        if self.stations[station_id].capacity <= (self.stations[station_id].level):
            self.Full_Error += 1
            yield self.Retrial(station_id)
        else:
            yield self.Bike_Arrival(station_id)

    def customer(self, id, station_id):
        # # # generate a Poisson interarrival time
        interarrival_time = np.random.exponential(15)

        # # wait for the next customer to arrive
        yield self.env.timeout(interarrival_time)

        station_id = random.randint(0, len(bike_share.stations)-1)
        # # Randomly select a station to rent a bike from
        #print(f"Customer {id} wants to rent a bike from station {station_id}.")

        # Try to rent a bike from the selected station
        yield env.process(bike_share.Customer_Arrival(station_id))
        print(f"Customer {id} has rented a bike from station {station_id}.")

        # Return Station
        #print(f"Customer {id} wants to return the bike to station {self.destination_id}.")

        # Try to return the bike to the selected station
        yield env.process(bike_share.Departure(self.destination_id))
        print(f"Customer {id} has RETURNED the bike to station {self.destination_id}")

    def run(self):
        while self.current_step < self.num_steps:
            # Execute some action at each time step
            print("Step:", self.current_step)
            customer_count = 0
            for i in range(10):
                customer_count += 1
                env.process(self.customer(customer_count, bike_share))
                # Increment the current step and wait for the next time step
            self.current_step += 0.5
            yield self.env.timeout(1)


env = simpy.Environment()
bike_share = BikeShareSystem(env, 0, 0, 2, 5, 0, 12)

# Run the simulation
env.process(bike_share.run())
env.run()
print("Full Error Count:", bike_share.Full_Error)
print("Empty Error Count:", bike_share.Empty_Error)


Step: 8
Step: 8.5
Customer_Arrival
Customer 4 has rented a bike from station 0.
Departure
Step: 9.0
Customer_Arrival
Customer 2 has rented a bike from station 0.
Departure
Customer_Arrival
Customer 8 has rented a bike from station 1.
Departure
Customer_Arrival
Customer 4 has rented a bike from station 1.
Departure
Customer_Arrival
Customer 2 has rented a bike from station 1.
Departure
Customer_Arrival
Customer 8 has rented a bike from station 0.
Departure
Step: 9.5
Customer_Arrival
Customer 1 has rented a bike from station 1.
Departure
Customer_Arrival
Customer 8 has rented a bike from station 0.
Departure
Step: 10.0
Customer_Arrival
Customer 9 has rented a bike from station 0.
Departure
Step: 10.5
Customer_Arrival
Customer 4 has rented a bike from station 0.
Departure
Customer_Arrival
Customer 7 has rented a bike from station 1.
Departure
Customer_Arrival
Customer 1 has rented a bike from station 1.
Departure
Step: 11.0
Customer_Arrival
Customer 5 has rented a bike from station 1.
Dep

In [ ]:
"""
MGSC
"""

Stations = [7, 15]
Station_Capacity = [10, 20]

ZSimRNG = SimRNG.InitializeRNSeed()

Queue = SimClasses.FIFOQueue()
Wait = SimClasses.DTStat()
Bikes = SimClasses.Resource()

Station_1_Resource = []
Station_2_Resource = []

Station_1 = SimClasses.Resource()
Station_1.SetUnits(Stations[0])
Station_1_Resource.append(Station_1)
Queue_1 = SimClasses.FIFOQueue()

Station_2 = SimClasses.Resource()
Station_2.SetUnits(Stations[1])
TheResources.append(Bikes)
Station_2_Resource.append(Station_2)
Queue_2 = SimClasses.FIFOQueue()


Calendar = SimClasses.EventCalendar()

TheCTStats = []
TheDTStats = []
TheQueues = []
TheResources = []

TheDTStats.append(Wait)
TheQueues.append(Queue)
TheResources.append(Bikes)

NumOfBikes = sum(Stations)
Bikes.SetUnits(NumOfBikes)

MeanTBA = 0.5
MeanTR = 0.5
MeanST = 0.5

Phases = 3
RunLength = 50
WarmUp = 5

AllWaitMean = []
AllQueueMean = []
AllQueueNum = []
AllServerMean = []


Full_Error = 0
Empty_Error = 0


In [82]:
class Station:
    def __init__(self, station_id, level, capacity):
        self.station_id = station_id
        self.level = level
        self.capcity = capacity
        
s1 = Station(station_id=1, level=10, capacity=20)
s1.level = 3
s1.capcity = 10
s1.level < s1.capcity

True

In [77]:
global Stations, Station_Capacity
Stations = [7, 15]
Station_Capacity = [10, 20]


class Station:
    def __init__(self, station_id, level, capacity):
        self.id = station_id
        self.level = level
        self.capacity = capacity

    def rent_bike(self):
        # Request a bike from the station
        self.level -= 1

    def return_bike(self):
        # Return a bike to the station
        self.level += 1
    
    def get_level(station):
        return station.level

class Trip:
    def __init__(self, station_id, destination_id, Start_S_level, End_S_level):
        self.start_id = station_id
        self.end_id = destination_id

        self.Start_S_level = Start_S_level
        #self.Start_S_capcity = Start_S_capacity

        self.End_S_level = End_S_level
        #self.End_S_capcity = End_S_capacity


In [78]:
    
def Departure(station, station_id):
    print("Departure")
    station.id = station_id
    
    Trip.id = station_id
    Trip.Start_S_level = station.level
    
    SimFunctions.Schedule(Calendar, "Bike_Arrival", SimRNG.Expon(1, 3))
    

def Trip():
    print("Trip")
    SimFunctions.Schedule(Calendar, "Trip", 15*SimRNG.lognormal(0.1, np.sqrt(0.066), 4))


def Destination(station):
    print("Destination")
    Trip.start_id = station.id

    U = np.random.uniform(0, 1)
    D = 1 if U > 1/2 else 0
    
    Trip.destination_id = D
    Trip.End_S_level = station.level

def Customer_Arrival(station, Empty_Error):
    print("Customer_Arrival")
    SimFunctions.Schedule(Calendar, "Customer_Arrival", SimRNG.Expon(0.5, 1))

    # STATION EMPTY
    if station.level <= 0:
        Empty_Error += 1
    else:
        station.rent_bike()
        Destination(station)
        SimFunctions.Schedule(Calendar, "Departure",  np.random.lognormal(0.1, np.sqrt(0.066)))

        #Departure(station, station.id)
        #Bike_Arrival(station, Full_Error)
    return station.id

def Bike_Arrival(station, Full_Error):
    print("Bike Arrival")
    SimFunctions.Schedule(Calendar, "Bike_Arrival", SimRNG.Expon(1, 3))
    
    # STATION FULL
    if station.level >= station.capacity:
        Full_Error += 1

    if station.level < station.capacity:
        station.return_bike()


def Retrial(station, Full_Error): 
    print("Retrial")
    SimFunctions.Schedule(Calendar, "Retrail", SimRNG.Expon(1, 5))
    # STATION FULL
    if station.level >= station.capacity:
        Full_Error += 1
        Retrial(station)
    else:
        Bike_Arrival(station)
        

In [79]:

"""
MGSC
"""
ZSimRNG = SimRNG.InitializeRNSeed()

Calendar = SimClasses.EventCalendar()

Queue = SimClasses.FIFOQueue()
Wait = SimClasses.DTStat()
Bikes = SimClasses.Resource()

TheCTStats = []
TheDTStats = []
TheQueues = []
TheResources = []

Stations = [7, 15]
Station_Capacity = [10, 20]

Station_1_Resource = []
Station_2_Resource = []

MeanTBA = 0.5
MeanTR = 0.5
MeanST = 0.5

Phases = 3
RunLength = 50
WarmUp = 5

for reps in range(0, 1, 1):
    Full_Error = 0
    Empty_Error = 0

    station = np.random.random_integers(0, 1)
    
    station_1 = Station(station_id=1, level=7, capacity=10)
    station_2 = Station(station_id=2, level=15, capacity=20)
    trip = Trip(station_id=1, destination_id=2, Start_S_level = station_1.level, End_S_level = station_2.level)

    SimFunctions.SimFunctionsInit(Calendar, TheQueues, TheCTStats, TheDTStats, TheResources)
    SimFunctions.Schedule(Calendar, "Customer_Arrival",SimRNG.Expon(MeanTBA, 1))
    SimFunctions.Schedule(Calendar, "EndSimulation", RunLength)
    SimFunctions.Schedule(Calendar, "ClearIt", WarmUp)

    NextEvent = Calendar.Remove()
    SimClasses.Clock = NextEvent.EventTime
    if NextEvent.EventType == "Customer_Arrival":
        Customer_Arrival(station_1, Empty_Error, Full_Error)

    for time in [8.0, 8.5]:#, 9.0, 9.5, 10.0, 10.5, 11.0, 11.5, 12.0]:  # 30 min intervals
        for time in [i for i in range(1, 31)]:  # 30 min intervals
            NextEvent = Calendar.Remove()
            SimClasses.Clock = NextEvent.EventTime

            if NextEvent.EventType == "Customer_Arrival":
                station = Customer_Arrival(station, Empty_Error, Full_Error)

            elif NextEvent.EventType == "Departure":
                Departure(station)
            
            elif NextEvent.EventType == "Trip":
                Trip()
            
            elif NextEvent.EventType == "Bike_Arrival":
                Bike_Arrival(station, Full_Error)
            
            elif NextEvent.EventType == "Retrial":
                Retrial(station, Full_Error)

            elif time == 12.0:
                break
            
            print("Station ID:", station.id)
            print("Station Level {} | Capacity {}".format( station.level, station.capacity))
            print("-------------------------------------------------------------------------")
            print("Trip Start ID: {} | End ID: {}".format(trip.id, trip.end))
            print("Station Level {} | Capacity {}".format( station.level, station.capacity))

Customer_Arrival
Destination
Departure
Bike Arrival
Bike Arrival


AttributeError: 'numpy.int32' object has no attribute 'level'